**Table of contents**<a id='toc0_'></a>    
- [移動ロボット](#toc1_)    
- [pybulletの起動](#toc2_)    
- [初期設定](#toc3_)    
- [パラメータなどの設定](#toc4_)    
- [オドメトリの推定](#toc5_)    
- [（おまけ）摩擦の変更](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[移動ロボット](#toc0_)



（pybulletで使用可能な関数がまとめられたマニュアルについては[こちら](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf)を参照してください。）

# <a id='toc2_'></a>[pybulletの起動](#toc0_)

In [1]:
import time
import pybullet
import pybullet_data
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


# <a id='toc3_'></a>[初期設定](#toc0_)

床の生成、ロボットの生成、カメラ位置の設定などの初期設定を行います。

In [2]:
pybullet.resetSimulation() # シミュレーション空間をリセット
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # pybulletに必要なデータへのパスを追加
pybullet.setGravity(0.0, 0.0, -9.8) # 地球上における重力に設定
time_step = 1./240.
pybullet.setTimeStep(time_step)

#床の読み込み
plane_id = pybullet.loadURDF("plane.urdf")

# ロボットの読み込み
car_start_pos = [0, 0, 0.1]  # 初期位置(x,y,z)を設定
car_start_orientation = pybullet.getQuaternionFromEuler([0,0,0])  # 初期姿勢(roll, pitch, yaw)を設定
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf",car_start_pos, car_start_orientation)

# GUIモードの際のカメラの位置などを設定
camera_distance = 2.0
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [0.0, 0.0, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

ven = Mesa
ven = Mesa


In [3]:
import math

class WheelOdometry:
    def __init__(self, wheel_radius, wheel_tread, const_right_wheel, const_left_wheel, initial_angle_right_wheel, initial_angle_left_wheel):
        # ホイールのサイズに関する定数
        self.WHEEL_DIAMETER = wheel_radius * 2
        self.WHEEL_TREAD = wheel_tread

        # 環境やタイヤの材質などを考慮した定数
        self.CONSTANT_RIGHT_WHEEL = const_right_wheel
        self.CONSTANT_LEFT_WHEEL = const_left_wheel

        # 1回転あたりの距離
        self.ONE_REVOLUTION_DISTANCE_RIGHT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_LEFT_WHEEL
        self.ONE_REVOLUTION_DISTANCE_LEFT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_RIGHT_WHEEL

        # ロボットの初期位置と姿勢 (x, y, theta)
        self.x = 0.0
        self.y = 0.0
        self.theta = 0.0

        # 前回のホイール角度
        self.last_angle_right_wheel = initial_angle_right_wheel
        self.last_angle_left_wheel = initial_angle_left_wheel

    def update_wheel_angles(self, current_angle_right_wheel, current_angle_left_wheel):
        # 左右のホイールの「回転角度の微小変化量[ΔΘ]」を計算
        delta_angle_right_wheel = current_angle_right_wheel - self.last_angle_right_wheel
        delta_angle_left_wheel = current_angle_left_wheel - self.last_angle_left_wheel

        # 左右のホイールの「回転角度の微小変化量[Δθ]」から、「回転数」を計算（2π → 1回転 となるように変換）
        revolution_right_wheel = delta_angle_right_wheel / (2.0 * math.pi)
        revolution_left_wheel = delta_angle_left_wheel / (2.0 * math.pi)

        # 左右のホイールが「移動した距離[m]」を計算
        distance_right_wheel = revolution_right_wheel * self.ONE_REVOLUTION_DISTANCE_RIGHT
        distance_left_wheel = revolution_left_wheel * self.ONE_REVOLUTION_DISTANCE_LEFT

        # 左右の移動距離から、「平均移動距離[m]」を計算
        distance_avg = (distance_right_wheel + distance_left_wheel) / 2.0

        # ロボットの姿勢（θ）を更新
        new_theta = self.theta + math.atan2(distance_right_wheel - distance_left_wheel, self.WHEEL_TREAD)

        # thetaを -π から π の範囲に収める
        if new_theta > math.pi:
            new_theta -= 2 * math.pi
        elif new_theta < -math.pi:
            new_theta += 2 * math.pi

        # ロボットの位置(x, y)を更新
        self.x += distance_avg * math.cos(new_theta)
        self.y += distance_avg * math.sin(new_theta)
        self.theta = new_theta

        # 現在のホイール角度を保存
        self.last_angle_right_wheel = current_angle_right_wheel
        self.last_angle_left_wheel = current_angle_left_wheel

    def get_position(self):
        # ロボットの現在の位置と姿勢を返す
        return self.x, self.y, self.theta




# <a id='toc4_'></a>[パラメータなどの設定](#toc0_)

In [4]:
# 左右の車輪のジョイントのインデックス
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# 車輪に関する定数
WHEEL_RADIUS = 0.05 # 車輪の半径(「simple_two_wheel_car.urdf」の車輪の半径と一致させる)
WHEEL_THREAD = 0.325 # 車輪の間隔(「simple_two_wheel_car.urdf」の車輪の間隔と一致させる)
CONST_RIGHT_WHEEL = 1.0 # 右車輪の定数（摩擦や材質などを考慮した定数）
CONST_LEFT_WHEEL = 1.0 # 左車輪の定数（摩擦や材質などを考慮した定数）

# 左右の車輪の速度を変更するスライダーを作成
pybullet.addUserDebugParameter("right_wheel_velocity", -10, 10, 0)
pybullet.addUserDebugParameter("left_wheel_velocity", -10, 10, 0)

1

# <a id='toc5_'></a>[オドメトリの推定](#toc0_)

シミュレーションを実行すると、右側に左右の車輪の速度を変更するためのスライダーを変更することで、ロボットを動かすことができます。

また、画面上に「真のロボットの位置」と「オドメトリによって推定されたロボットの位置」が表示されます。

In [6]:
import sys

# debug用の描画を初期化
pybullet.removeAllUserDebugItems()
replace_pose_texts = []

# ロボットを初期位置にセット
car_start_pos = [0, 0, 0.1]
car_start_orientation = pybullet.getQuaternionFromEuler([0.0, 0.0, 0.0])
pybullet.resetBasePositionAndOrientation(car_id, car_start_pos, car_start_orientation)

# 左右の車輪の回転角度の初期値を取得
initial_angle_right_wheel = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
initial_angle_left_wheel = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

# オドメトリクラスのインスタンス化
wheel_odometry = WheelOdometry(wheel_radius=WHEEL_RADIUS, wheel_tread=WHEEL_THREAD, const_right_wheel=CONST_RIGHT_WHEEL, const_left_wheel=CONST_LEFT_WHEEL, initial_angle_right_wheel=initial_angle_right_wheel, initial_angle_left_wheel=initial_angle_left_wheel)

enable_debug_text = False # PybulletのGUI画面上にデバッグ用のテキストを表示するかどうか（Trueにすると、処理が重く処理が重くなるので注意）

# スライダーの設定された値に応じて、移動ロボットを制御し、オドメトリを計算
while True:
    # スライダーの値を取得
    right_wheel_velocity = pybullet.readUserDebugParameter(0)
    left_wheel_velocity = pybullet.readUserDebugParameter(1)

    # 2輪を前進するように速度制御
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # 現在の車輪の回転角度を取得
    current_angle_wheel_right = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
    current_angle_wheel_left = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

    # オドメトリの計算
    wheel_odometry.update_wheel_angles(current_angle_wheel_right, current_angle_wheel_left)
    odom_x, odom_y, odom_theta = wheel_odometry.get_position()

    # 真の位置・姿勢を取得
    true_car_pos, true_car_orientation = pybullet.getBasePositionAndOrientation(car_id)
    true_x = true_car_pos[0]
    true_y = true_car_pos[1]
    true_euler = pybullet.getEulerFromQuaternion(true_car_orientation)
    true_theta = true_euler[2]
    
    # pybulletの画面上に推定した位置・姿勢と実際の位置・姿勢を表示
    if enable_debug_text:
        if len(replace_pose_texts) == 0:
            replace_pose_texts.append(pybullet.addUserDebugText(f"Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}", [-1.0, -0.5, 0.0], textColorRGB=[1,0,0], textSize=1.5))
            replace_pose_texts.append(pybullet.addUserDebugText(f"Ture Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta):.2f}", [-1.0, -0.7, 0.0], textColorRGB=[0,1,0], textSize=1.5))
        else:
            pybullet.addUserDebugText(f"Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}", [-1.0, -0.5, 0.0], textColorRGB=[1,0,0], textSize=1.5, replaceItemUniqueId=replace_pose_texts[0])
            pybullet.addUserDebugText(f"Ture Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta):.2f}", [-1.0, -0.7, 0.0], textColorRGB=[0,1,0], textSize=1.5, replaceItemUniqueId=replace_pose_texts[1])
    
    # print文でも同様に表示
    sys.stdout.write(f"\r【Estimate Pose x:{odom_x:.2f}, y:{odom_y:.2f}, theta:{math.degrees(odom_theta):.2f}】【True Pose x:{true_x:.2f}, y:{true_y:.2f}, theta:{math.degrees(true_theta):.2f}】                       ")

    time.sleep(time_step)


【Estimate Pose x:-0.49, y:-0.02, theta:19.00】【True Pose x:-0.46, y:0.04, theta:13.85】                       

KeyboardInterrupt: 

# <a id='toc6_'></a>[（おまけ）摩擦の変更](#toc0_)

以下コードを実行することで、ロボットの摩擦係数を変更することができます。

摩擦係数を変更した後に、上記コードをもう一度実行すると、オドメトリの推定結果が変わることが確認できます。

In [ ]:
plane_lateral_friction = 1.0 # 床の摩擦係数（デフォルトは1.0）
wheel_lateral_friction = 1.0 # ホイールの摩擦係数（デフォルトは1.0）

# 地面の摩擦を変更
pybullet.changeDynamics(plane_id, -1, lateralFriction=plane_lateral_friction)

# ロボットのホイール摩擦を変更
for wheel_link in range(pybullet.getNumJoints(car_id)):
    pybullet.changeDynamics(car_id, wheel_link, lateralFriction=wheel_lateral_friction)